In [1]:
!pip install aicrowd-cli==0.1
API_KEY = "4c49d255257272f8caf90b1e74b8cccd"
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [2]:
!ls

data	     sample_submission.csv  train.csv  val.csv
sample_data  test.zip		    train.zip  val.zip


In [3]:
!aicrowd dataset download --challenge rover-classification -j 3

sample_submission.csv: 100% 164k/164k [00:00<00:00, 1.99MB/s]
train.zip:   0% 0.00/266M [00:00<?, ?B/s]
train.csv:   0% 0.00/689k [00:00<?, ?B/s]
train.csv: 100% 689k/689k [00:00<00:00, 4.76MB/s]

val.csv: 100% 64.8k/64.8k [00:00<00:00, 1.40MB/s]
train.zip:  13% 33.6M/266M [00:00<00:03, 61.5MB/s]
train.zip:  25% 67.1M/266M [00:00<00:02, 68.5MB/s]
val.zip: 100% 26.6M/26.6M [00:00<00:00, 56.1MB/s]

train.zip: 100% 266M/266M [00:03<00:00, 86.2MB/s]
train.zip: 100% 266M/266M [00:03<00:00, 86.1MB/s]
test.zip: 100% 66.4M/66.4M [00:06<00:00, 10.9MB/s]


In [4]:
!rm -rf data
!mkdir data


!unzip train.zip -d data/train >/dev/null
!unzip val.zip -d data/val >/dev/null
!unzip test.zip -d data/test >/dev/null

In [5]:
import pandas as pd
import os
import tensorflow as tf

In [6]:
df_train = pd.read_csv("train.csv")

In [7]:
df_val = pd.read_csv("val.csv")

In [8]:
df_train['ImageID'] = df_train['ImageID'].astype(str)+".jpg"
df_val['ImageID'] = df_val['ImageID'].astype(str)+".jpg"

In [9]:
INPUT_SIZE = 256

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
len(df_train), len(df_val)

(40000, 4000)

In [12]:
datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="data/train/",
x_col="ImageID",
y_col="label",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(INPUT_SIZE,INPUT_SIZE))

Found 40000 validated image filenames belonging to 2 classes.


In [13]:
val_generator=datagen.flow_from_dataframe(
dataframe=df_val,
directory="data/val/",
x_col="ImageID",
y_col="label",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(INPUT_SIZE,INPUT_SIZE))

Found 4000 validated image filenames belonging to 2 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers, optimizers
import os
import numpy as np
import pandas as pd

In [15]:
class CustomAugment(object):
    def __call__(self, image):        
        # Random flips and grayscale with some stochasticity
        img = self._random_apply(tf.image.flip_left_right, image, p=0.5)
        img = self._random_apply(self._color_drop, img, p=0.8)
        return img

    def _color_drop(self, x):
        image = tf.image.rgb_to_grayscale(x)
        image = tf.tile(x, [1, 1, 1, 3])
        return x
    
    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

In [16]:
data_augmentation = tf.keras.Sequential(
  [
     #tf.keras.layers.Lambda(CustomAugment()),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(INPUT_SIZE, 
                                                              INPUT_SIZE,
                                                              3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [17]:
model = Sequential()
model.add(data_augmentation)
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizers.RMSprop(lr=0.00001), loss="categorical_crossentropy", metrics=["Recall", "Precision"])

In [18]:
next(train_generator)[0].shape

(64, 256, 256, 3)

In [19]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VAL=val_generator.n//train_generator.batch_size

In [20]:
model.fit(train_generator, validation_data=val_generator, epochs=10)

Epoch 1/10
625/625 [==============================] - 120s 185ms/step - loss: 0.6922 - recall: 0.5440 - precision: 0.5440 - categorical_crossentropy: 0.6922 - val_loss: 0.6025 - val_recall: 0.6760 - val_precision: 0.6760 - val_categorical_crossentropy: 0.6025
Epoch 2/10
625/625 [==============================] - 113s 181ms/step - loss: 0.5678 - recall: 0.7299 - precision: 0.7299 - categorical_crossentropy: 0.5678 - val_loss: 0.4810 - val_recall: 0.7918 - val_precision: 0.7918 - val_categorical_crossentropy: 0.4810
Epoch 3/10
625/625 [==============================] - 113s 180ms/step - loss: 0.4714 - recall: 0.7781 - precision: 0.7781 - categorical_crossentropy: 0.4714 - val_loss: 0.4994 - val_recall: 0.7592 - val_precision: 0.7592 - val_categorical_crossentropy: 0.4994
Epoch 4/10
625/625 [==============================] - 113s 181ms/step - loss: 0.4151 - recall: 0.8063 - precision: 0.8063 - categorical_crossentropy: 0.4151 - val_loss: 0.5458 - val_recall: 0.7395 - val_precision: 0.7395

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 127, 127, 32)     

In [22]:
df_test = pd.read_csv("sample_submission.csv",dtype=str)
df_test["ImageID"] = df_test["ImageID"].astype(str)+".jpg"

In [23]:
test_generator=datagen.flow_from_dataframe(
dataframe=df_test,
directory="data/test/",
x_col="ImageID",
y_col="label",
batch_size=1,
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(INPUT_SIZE,INPUT_SIZE))

Found 10000 validated image filenames belonging to 2 classes.


In [24]:
model

In [25]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

In [26]:
STEP_SIZE_TEST

10000

In [27]:
test_generator.reset()
pred = model.predict(test_generator,
steps=STEP_SIZE_TEST, verbose=1)

10000/10000 [==============================] - 35s 3ms/step


In [28]:
predicted_class_indices = np.argmax(pred,axis=1)

In [29]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [30]:
df_test["pred"] = predictions

In [31]:
df_test.head()

,ImageID,label,pred
0,0.jpg,curiosity,curiosity
1,1.jpg,perseverance,curiosity
2,2.jpg,curiosity,curiosity
3,3.jpg,perseverance,perseverance
4,4.jpg,perseverance,curiosity


In [32]:
df_test.drop("label", axis=1, inplace=True)
df_test.rename(columns={"pred": "label"}, inplace=True)

In [33]:
df_test.head()

,ImageID,label
0,0.jpg,curiosity
1,1.jpg,curiosity
2,2.jpg,curiosity
3,3.jpg,perseverance
4,4.jpg,curiosity


In [34]:
df_test.to_csv("data/01_sub.csv", index=False)

In [35]:
!aicrowd submission create -c rover-classification -f data/01_sub.csv

01_sub.csv ━━━━━━━━━━━━━━━━━━━━━━ 100.0% • 200.7/199.0 KB • 628.1 kB/s • 0:00:00
                                    ╭─────────────────────────╮                                     
                                    │ Successfully submitted! │                                     
                                    ╰─────────────────────────╯                                     
                                          Important links                                           
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-7/submissions/126499              │
│                  │                                                                               │
│  All submissions │ https://www.aicrowd.com/challenges/ai-blitz-7/submissions?my_submissions=true │
│                  │                                                                               │
│      Lea